### Installing CBC solver in google colab

In [19]:
!apt-get install -y -qq coinor-cbc

Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 144786 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coinor-libcgl1 (0.59.10+repack1-1) ...
Selecting previously unselected package coinor-libcbc3.
Preparing to unpack .../4-coinor-libcbc3_2.9.9+repack1-1_amd64.deb ...
Unpacking coinor-libcbc3 (2.9.9+repack1-1) ...
Selecting p

Importing required libraries

In [39]:
import pandas as pd
import numpy as np
from pyomo.environ import *
from pyomo.opt import SolverFactory
pd.set_option('precision', 0)

### Creating sample sudoku grid

In [32]:
### Grid size
N = list(range(1, 9 + 1))

### Creating grid
completeGrid = []

for i in N:
    for j in N:
        completeGrid.append((i,j))
        
completeGrid = pd.DataFrame(completeGrid, columns = ['i', 'j'])

### Known cell values
data = [(1, 7, 2), (2, 2, 8), (2, 6, 7), (2, 8, 9),
		(3, 1, 6), (3, 3, 2), (3, 7, 5), (4, 2, 7),
		(4, 5, 6), (5, 4, 9), (5, 6, 1), (6, 5, 2), 
		(6, 8, 4), (7, 3, 5), (7, 7, 6), (7, 9, 3),
		(8, 2, 9), (8, 4, 4), (8, 8, 7), (9, 3, 6)]

data = pd.DataFrame(data, columns = ['i', 'j', 'k'])

###
finalGrid = pd.merge(completeGrid, data, how = 'left')
finalGrid = finalGrid.fillna(0)

In [37]:
### Observing data
def color_red(val):
    color = 'red' if val != 0 else 'black'
    return 'color: %s' % color

finalGrid.pivot(index = 'i', columns = 'j', values = 'k').style.applymap(color_red)

j,1,2,3,4,5,6,7,8,9
i,,,,,,,,,
1,0,0,0,0,0,0,2,0,0
2,0,8,0,0,0,7,0,9,0
3,6,0,2,0,0,0,5,0,0
4,0,7,0,0,6,0,0,0,0
5,0,0,0,9,0,1,0,0,0
6,0,0,0,0,2,0,0,4,0
7,0,0,5,0,0,0,6,0,3
8,0,9,0,4,0,0,0,7,0
9,0,0,6,0,0,0,0,0,0


### Model and constraint definition

In [26]:
### Defining model
model = ConcreteModel()
model.X = Var(N, N, N, within = Binary)

### Definding constraints

## Row constraint
model.row_constr = ConstraintList()
for i in N:
    for k in N:
        model.row_constr.add(1 == sum(model.X[i, j, k] for j in N))
        
## Column constraint
model.col_constr = ConstraintList()
for j in N:
    for k in N:
        model.col_constr.add(1 == sum(model.X[i, j, k] for i in N))
        
## Block constraint
model.block_constr = ConstraintList()
for i in list(range(1, 9, 3)):
    for j in list(range(1, 9, 3)):
        for k in N:
            model.block_constr.add(1 == sum(model.X[p, q, k] for p in range(i, i+3) for q in range(j, j+3)))
            
## Constraint for assigning all cells
model.allCell_constr = ConstraintList()
for i in N:
    for j in N:
        model.allCell_constr.add(1 == sum(model.X[i, j, k] for k in N))
        
## Constraint for assigning known cells
model.knownCell_constr = ConstraintList()
for i, j, k in zip(finalGrid['i'], finalGrid['j'], finalGrid['k']):
    if k != 0:
        #print(k)
        model.knownCell_constr.add(1 == model.X[i,j,k])
        
## Defining objective function
model.Obj = Objective(expr = 1)

## Solving 
opt = SolverFactory('cbc', executable='/usr/bin/cbc')
opt.solve(model)

    solver failure.


{'Problem': [{'Name': 'unknown', 'Lower bound': 1.0, 'Upper bound': 1.0, 'Number of objectives': 1, 'Number of constraints': 172, 'Number of variables': 182, 'Number of binary variables': 729, 'Number of integer variables': 729, 'Number of nonzeros': 0, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'User time': -1.0, 'System time': 0.03, 'Wallclock time': 0.03, 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}, 'Black box': {'Number of iterations': 0}}, 'Error rc': 0, 'Time': 0.04693889617919922}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

### Running solver

In [27]:
solutionDf = pd.DataFrame({'i': [i[0] for i in model.X.iterkeys()], 'j': [i[1] for i in model.X.iterkeys()], 'k': [i[2] for i in model.X.iterkeys()], 'assignment': [i.value for i in model.X.itervalues()]})
solutionDf = solutionDf[solutionDf['assignment'] == 1]

In [28]:
solutionDf.pivot(index = 'i', columns = 'j', values = 'k')

j,1,2,3,4,5,6,7,8,9
i,,,,,,,,,
1,9,5,7,6,1,3,2,8,4
2,4,8,3,2,5,7,1,9,6
3,6,1,2,8,4,9,5,3,7
4,1,7,8,3,6,4,9,5,2
5,5,2,4,9,7,1,3,6,8
6,3,6,9,5,2,8,7,4,1
7,8,4,5,7,9,2,6,1,3
8,2,9,1,4,3,6,8,7,5
9,7,3,6,1,8,5,4,2,9
